# 3_EDA: Price Predictor

In [2]:
# Imports
import pandas as pd

In [3]:
# Read file cleaned in DW
supply_chain_df_cleaned = pd.read_csv('../../data/price_predictor/staged/Sample_Product_Data_Cleaned.csv')

---

### Basic data exploration

In [6]:
# Data glance
supply_chain_df_cleaned.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,cosmetics,062000007,22.79,17.0,715.0,8656.12,Non-binary,58.0,24.0,79.0,...,Kolkata,16.0,168.0,21.0,71.42,Pending,3.35,Rail,Route_B,407.65
1,cosmetics,1VBA0260GE,53.97,78.0,949.0,4516.14,Unknown,75.0,15.0,90.0,...,Delhi,3.0,617.0,28.0,80.63,Pass,4.66,Rail,Route_A,424.80
2,skincare,1VBF0810GE,80.18,44.0,687.0,6449.54,Male,97.0,20.0,10.0,...,Delhi,13.0,306.0,17.0,65.34,Pending,0.61,Air,Route_B,603.41
3,skincare,1VMG0080GE,62.87,46.0,903.0,1755.28,Female,18.0,21.0,98.0,...,Delhi,20.0,486.0,29.0,29.09,Fail,3.09,Rail,Route_C,331.07
4,skincare,1VMG0140GE,83.82,99.0,163.0,7097.87,Non-binary,88.0,18.0,50.0,...,Chennai,23.0,941.0,17.0,97.18,Pending,2.58,Road,Route_B,392.25


In [9]:
# Dataset info
supply_chain_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             1790 non-null   object 
 1   SKU                      1790 non-null   object 
 2   Price                    1790 non-null   float64
 3   Availability             1790 non-null   float64
 4   Number of products sold  1790 non-null   float64
 5   Revenue generated        1790 non-null   float64
 6   Customer demographics    1790 non-null   object 
 7   Stock levels             1790 non-null   float64
 8   Lead times               1790 non-null   float64
 9   Order quantities         1790 non-null   float64
 10  Shipping times           1790 non-null   float64
 11  Shipping carriers        1790 non-null   object 
 12  Shipping costs           1790 non-null   float64
 13  Supplier name            1790 non-null   object 
 14  Location                

---

### Dataset description

In [11]:
# Dataset info
supply_chain_df_cleaned.describe()

,Price,Availability,Number of products sold,Revenue generated,Stock levels,Lead times,Order quantities,Shipping times,Shipping costs,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Defect rates,Costs
count,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000,1790.000000
mean,50.966804,50.641341,508.297765,5594.913486,49.981006,14.745810,49.790503,5.062011,5.497246,14.970950,537.507821,15.179330,55.422536,2.508626,550.428497
std,28.349810,28.572735,285.985693,2569.747229,28.676248,8.347382,28.090631,2.561280,2.557734,8.296305,259.669904,8.213523,25.849102,1.438392,257.798706
min,1.010000,1.000000,1.000000,1016.190000,1.000000,1.000000,1.000000,1.000000,1.010000,1.000000,100.000000,1.000000,10.000000,0.000000,100.770000
25%,26.040000,26.000000,263.000000,3403.432500,25.000000,7.000000,26.000000,3.000000,3.260000,8.000000,317.500000,8.000000,33.065000,1.290000,324.517500
50%,51.370000,51.000000,516.000000,5640.640000,51.000000,15.000000,50.000000,5.000000,5.430000,15.000000,534.500000,15.000000,55.855000,2.500000,557.175000
75%,75.395000,75.000000,756.000000,7758.960000,75.000000,22.000000,75.000000,7.000000,7.680000,22.000000,763.000000,22.000000,77.797500,3.777500,766.612500
max,99.990000,99.000000,999.000000,9994.800000,99.000000,29.000000,99.000000,9.000000,10.000000,29.000000,999.000000,29.000000,99.880000,5.000000,998.960000


| **Variable Name**          | **Description**                                    | **Type of Data** |
|----------------------------|----------------------------------------------------|------------------|
| Product type               | Type of product                                    | Object           |
| SKU                        | Stock Keeping Unit                                 | Object           |
| Price                      | Price of the product                               | Float            |
| Availability               | Availability status of the product                 | Float            |
| Number of products sold    | Number of products sold                            | Float            |
| Revenue generated          | Total revenue generated                            | Float            |
| Customer demographics      | Demographic details of the customers               | Object           |
| Stock levels               | Current stock levels                               | Float            |
| Lead times                 | Time taken from order to delivery                  | Float            |
| Order quantities           | Quantities ordered                                 | Float            |
| Shipping times             | Time taken for shipping                            | Float            |
| Shipping carriers          | Carriers used for shipping                         | Object           |
| Shipping costs             | Cost incurred for shipping                         | Float            |
| Supplier name              | Name of the supplier                               | Object           |
| Location                   | Location of the supplier                           | Object           |
| Lead time                  | Time taken by supplier to fulfill orders           | Float            |
| Production volumes         | Volume of products produced                        | Float            |
| Manufacturing lead time    | Time taken for manufacturing                       | Float            |
| Manufacturing costs        | Costs involved in manufacturing                    | Float            |
| Inspection results         | Results from product inspections                   | Object           |
| Defect rates               | Rates of defects found in products                 | Float            |
| Transportation modes       | Modes of transportation used                       | Object           |
| Routes                     | Routes used for transportation                     | Object           |
| Costs                      | Overall costs                                      | Float            |


# TO-DO:

## 3. Get Insights for Each Column
- Display descriptive statistics for numerical columns.
- Display unique values and frequency counts for categorical columns.
- Summarize key insights from each column.

## 4. Visualize Distributions of Numerical Data
- Plot histograms and boxplots for each numerical column to understand their distributions.
- Identify potential outliers and understand their impact.

## 5. Visualize Categorical Data
- Plot bar charts for categorical columns to understand their distributions.
- Analyze the balance of categories.

## 7. Check Data Distributions for Machine Learning Validation (Regression Model)
- Check which features to standarize or normalize (standarization and normalization are applied in feature_engineering.ipynb)
- Check for skewness and identify which transformations are relevant (transformations are applied in feature_engineering.ipynb).
- Check which categorical features are needed to one hot encode (one hot encode is applied in feature_engineering.ipynb).

## 8. Check Which Features Can Be Useful for Modeling
- Identify potential predictors for the target variable (price)
- Compute and visualize the correlation matrix using heatmaps.
- Identify multicollinearity using Variance Inflation Factor (VIF).
- Identify which columns to remove or combine that are highly correlated to prevent multicollinearity (removal takes place in feature_engineering.ipynb)
- Minimally document all findings and steps taken during the EDA
- Create fancy visualizations for the steps taken

**OUTPUT:** None